Skive Vision

In [146]:
# Initializations

In [147]:
import os
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel
import pandas as pd
from typing import Type
from pydantic import Field
import re
import dotenv

dotenv.load_dotenv()

model = ChatAnthropic(model="claude-3-opus-20240229", anthropic_api_key=os.environ["ANTHROPIC_API_KEY"])

KeyError: 'ANTHROPIC_API_KEY'

In [ ]:
# Utility functions : Convert PDF pages to images

In [ ]:
def pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)

def image_to_text(image: Image.Image):
    return pytesseract.image_to_string(image)

In [ ]:
# Image to Text Conversion Tool

In [ ]:
class ImageToTextInputs(BaseModel):
    image_path: str = Field(..., description="Path to the image file")

class ImageToTextTool(StructuredTool):
    name = "ImageToText"
    description = "Convert an image to text using OCR. Input should be the path to the image file."
    args_schema: Type[ImageToTextInputs] = ImageToTextInputs

    def _run(self, image_path: str) -> str:
        image = Image.open(image_path)
        return image_to_text(image)

image_to_text_tool = ImageToTextTool()

In [ ]:
# Information Verification Tool

In [ ]:
class VerifyInfoInputs(BaseModel):
    text: str
    required_info: list

def verify_info(text: str, required_info: list):
    return all(info.lower() in text.lower() for info in required_info)

class VerifyInfoTool(StructuredTool):
    name = "VerifyInfo"
    description = "Verify if the required information is present in the text."
    args_schema: Type[VerifyInfoInputs] = VerifyInfoInputs

    def _run(self, text: str, required_info: list) -> bool:
        return verify_info(text, required_info)

verify_info_tool = VerifyInfoTool()

In [ ]:
# Extract Info Tool

In [ ]:
class ExtractInfoInputs(BaseModel):
    text: str
    fields: list

def extract_info(text: str, fields: list):
    extracted_info = {}
    for field in fields:
        start = text.lower().find(field.lower())
        if start != -1:
            end = text.find('\n', start)
            if end == -1:
                end = len(text)
            value = text[start + len(field):end].strip(': ').strip()
            extracted_info[field] = value
        else:
            extracted_info[field] = None
    return extracted_info

class ExtractInfoTool(StructuredTool):
    name = "ExtractInfo"
    description = "Extract required information from the text."
    args_schema: Type[ExtractInfoInputs] = ExtractInfoInputs

    def _run(self, text: str, fields: list) -> dict:
        return extract_info(text, fields)

extract_info_tool = ExtractInfoTool()


In [ ]:
# LLM Chains

In [ ]:
pdf_processing_chain = LLMChain(
    llm=model,
    prompt=ChatPromptTemplate.from_template("Extract text from the image: {image}")
)

verification_chain = LLMChain(
    llm=model,
    prompt=ChatPromptTemplate.from_template("Verify the extracted text. Required information: {required_info}")
)

extraction_chain = LLMChain(
    llm=model,
    prompt=ChatPromptTemplate.from_template("Extract the following fields from the text: {fields}")
)

In [ ]:
# Process PDF's 

In [ ]:
def process_pdf(pdf_path, required_info, fields, output_folder="output_images"):
    try:
        os.makedirs(output_folder, exist_ok=True)
        
        images = pdf_to_images(pdf_path)
        all_extracted_info = []
        document_valid = False  # Flag to track overall document validity

        for i, image in enumerate(images):
            image_path = os.path.join(output_folder, f"image_{i}.png")
            image.save(image_path)
            
            try:
                extracted_text = image_to_text_tool.run({"image_path": image_path})
                
                # Verify if all required info is present
                verification_result = verify_info_tool.run({
                    "text": extracted_text, 
                    "required_info": required_info
                })

                extraction_result = extract_info_tool.run({
                    "text": extracted_text, 
                    "fields": fields
                })
                
                # Add verification result to extraction_result
                extraction_result['Verification'] = 'Valid' if verification_result else 'Invalid'
                
                all_extracted_info.append(extraction_result)
                
                # Update overall document validity
                if verification_result:
                    document_valid = True

            except Exception as e:
                print(f"Error processing image {i}: {str(e)}")
                print(f"Error type: {type(e)}")
                print(f"Error details: {e.args}")

        return all_extracted_info, document_valid
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return [], False

In [ ]:
# Test from here

In [ ]:
pdf_path = r"C:\Users\USER\Documents\GitHub\skive.ai\Docs\BGQPK4512E_12022024032132 (1).pdf"
required_info = ["Name", "Date", "Address"]
fields = ["Name", "Date", "Address"]
output_folder = "output_images"
extracted_info, document_valid = process_pdf(pdf_path, required_info, fields, output_folder)

df = pd.DataFrame(extracted_info)
print(df)

print("\nDocument Validation Result:")
print("Certified Valid" if document_valid else "Not Certified Valid")

                                                 Name  \
0                    * (same as ID proof) MANOJ KUMAR   
1                                         MANOJ KUMAR   
2                                                       
3                                         MANOJ KUMAR   
4                                         MANOJ KUMAR   
5     of the Trading Member: INDmoney Private Limited   
6   of the document Brief description of the document   
7   & address of the applicant mentioned on the KY...   
8                      of the spouse may be accepted.   
9                            of Applicant MANOJ KUMAR   
10                                        MANOJ KUMAR   
11                                                are   
12                                        MANOJ KUMAR   
13                           as per following details   
14                                   INDmoney Pvt Ltd   
15                                                      
16                             